# Combining the two sets of data logged from the SSN website 

Having previously cleaned the datasets ([1](./ExamineANMReadings.ipynb), [2](./ExamineElectricityReadings.ipynb)), such that they have a maximum of one record in any minute period, they now need to be combined into a single dataset such that for each data point, there exists a complete record of electricity readings and 'ANM' status.

This process is complicated due to the ANM records having been stored only when there is a change, as opposed to every reading.  This was done to reduce the overall number of records stored and consequently, minimise storage requirements. There is no lost data as the duration of any particular event can be inferred from the start time of the subsequent one.   

Two methods are examined here:
  * combining the tables into a single database to run a single SQL query
  * using a python script to run through each entry in the readings table and pairing it with the appropriate entry in the ANM log

A script has also been written to parse the JSON format logged data and combine it into a single database.

---

## Initialise working environment

* load iPython  %sql magic extension  
* connect to databases and check they are as expected

In [25]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Electricity log database
---

In [4]:
%sql sqlite:///./database/eleclog_copy.db

'Connected: @./database/eleclog_copy.db'

In [4]:
%sql SELECT COUNT(*) FROM readings;

 * sqlite:///./database/eleclog_copy.db
Done.


COUNT(*)
395525


In [5]:
%sql SELECT sql FROM sqlite_master WHERE name = 'readings';

 * sqlite:///./database/eleclog_copy.db
Done.


sql
"CREATE TABLE readings (timestamp DATETIME, demand NUMERIC, demand_max NUMERIC, anm_generated NUMERIC, non_anm_generated NUMERIC, total_generation_capacity NUMERIC)"


In [5]:
#copy readings table to a local variable
readings_orig = %sql select * from readings order by timestamp;

 * sqlite:///./database/eleclog_copy.db
Done.


###  ANM event database
---

In [18]:
%sql sqlite:///./database/ANMlog_copy.db

'Connected: @./database/ANMlog_copy.db'

In [7]:
%sql SELECT COUNT(*) FROM records;

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
Done.


COUNT(*)
15711


In [12]:
%sql SELECT sql FROM sqlite_master WHERE name = 'records';

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
Done.


sql
"CREATE TABLE records(timestamp DATETIME, log TEXT)"


In [20]:
#copy readings table to a local variable
records = %sql select * from records order by timestamp;

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.


###  Create combined database
---

In [28]:
%sql sqlite:///./database/combined.db

'Connected: @./database/combined.db'

%sql magic provides for creation of tables directly from DataFrame types, using the 'persists' command

It is unecessary to create the tables in the database prior to loading the data
>
```python
%sql CREATE TABLE readings (timestamp DATETIME, demand NUMERIC, demand_max NUMERIC, anm_generated NUMERIC, non_anm_generated NUMERIC, total_generation_capacity NUMERIC);
%sql CREATE TABLE records(timestamp DATETIME, log TEXT);
```

*If the table already exists in the external database file the 'persist' function fails.  To speed up development, these commands have been commented out in the following cells.* 

In [17]:
records.DataFrame().head()

,timestamp,log
0,2019-01-16 22:42:33,"{""2"": [""warning"", ""ok"", ""ok""], ""2B"": [""warning..."
1,2019-01-16 22:46:25,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
2,2019-01-16 22:59:22,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
3,2019-01-16 23:03:05,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."
4,2019-01-16 23:06:26,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""..."


In [21]:
# -persist functions have been commented out as they are only needed to create the database file './database/combined.db'. (i.e. the first time this page is run) 

records = records.DataFrame()
# %sql persist records 

readings = readings_orig.DataFrame()
# %sql persist readings

---

## Joins

In [9]:
print(readings_orig[0], records[0])

('2019-01-16 14:36:03', 23.22, 35.7, 12.478, 17.782, 57.099999999999994) ('2019-01-16 22:42:33', '{"2": ["warning", "ok", "ok"], "2B": ["warning", "ok", "ok"], "2A": ["remove", "ok", "ok"], "4A": ["ok", "ok", "ok"], "3": ["ok", "ok", "ok"], "Core": ["ok", "ok", "ok"], "1": ["ok", "ok", "ok"], "4": ["ok", "ok", "ok"], "1A": ["ok", "ok", "ok"]}')


### Simple join matching records from ANM logs with electricity readings.

Using a substring of the timestamp matches records by the 'minute' avoiding incongruity with differing 'seconds'

However, it only matches each record from the ANM log with one from the electricity readings.  Hence a lagre number of electricity readings are omitted.

In [30]:
%%sql
select
    *
from
    readings
left join
    records
where
    substr(readings.timestamp, 0, 17) == substr(records.timestamp, 0, 17)
limit
    5
    ;

   sqlite:///./database/ANMlog_copy.db
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.


index,timestamp,demand,demand_max,anm_generated,non_anm_generated,total_generation_capacity,index_1,timestamp_1,log
470,2019-01-16 22:42:03,19.66,35.7,17.145,17.075,57.099999999999994,0,2019-01-16 22:42:33,"{""2"": [""warning"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""], ""1"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""]}"
474,2019-01-16 22:46:03,18.7,35.7,14.572,16.457,57.1,1,2019-01-16 22:46:25,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
487,2019-01-16 22:59:03,18.28,35.7,15.618,16.093,57.099999999999994,2,2019-01-16 22:59:22,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
491,2019-01-16 23:03:05,18.56,35.7,13.922,18.143,57.1,3,2019-01-16 23:03:05,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""ok"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
494,2019-01-16 23:06:03,18.2,35.7,16.457,17.996,57.099999999999994,4,2019-01-16 23:06:26,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"


Extract from AMN log the most recent two events for a particular time.

In [14]:
%%sql
select * from records where timestamp < '2019-01-17 09:35:11' order by timestamp desc limit 2;

 * sqlite:///./database/ANMlog_copy.db
   sqlite:///./database/eleclog_copy.db
Done.


timestamp,log
2019-01-17 09:34:10,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""warning"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"
2019-01-17 09:31:11,"{""1"": [""ok"", ""ok"", ""ok""], ""1A"": [""ok"", ""ok"", ""ok""], ""2"": [""remove"", ""ok"", ""ok""], ""2A"": [""remove"", ""ok"", ""ok""], ""2B"": [""warning"", ""ok"", ""ok""], ""3"": [""ok"", ""ok"", ""ok""], ""4"": [""ok"", ""ok"", ""ok""], ""4A"": [""ok"", ""ok"", ""ok""], ""Core"": [""ok"", ""ok"", ""ok""]}"


In [16]:
# match most recent ANM log event to a number of sample readings from  electricity log

for reading in readings_orig[30000:30010]:
    readingdate= "'" + reading['timestamp'] + "'"
    anmstatus = %sql select * from records where timestamp < $readingdate order by timestamp desc limit 1;
    print('electricity reading timestamp: ',reading['timestamp'],' | ANM event timestamp: ', anmstatus[0]['timestamp'])

 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
electricity reading timestamp:  2019-02-06 18:14:09  | ANM event timestamp:  2019-02-06 18:13:09
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
electricity reading timestamp:  2019-02-06 18:15:05  | ANM event timestamp:  2019-02-06 18:14:13
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
electricity reading timestamp:  2019-02-06 18:16:05  | ANM event timestamp:  2019-02-06 18:14:13
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
electricity reading timestamp:  2019-02-06 18:17:05  | ANM event timestamp:  2019-02-06 18:14:13
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
electricity reading timestamp:  2019-02-06 18:18:05  | ANM event timestamp:  2019-02-06 18:14:13
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
electricity reading tim

#### Comment

Although this appears to have worked as expected, notice that the first and second results indicate something amiss.

* In the first result(30000), the electricity reading was taken on '2019-02-06 18:14:09'  whilst the matched previous ANM event occurred on '2019-02-06 18:13:09'
* In the Second result (30001), the ANM reading was taken on '2019-02-06 18:14:13' - only a few seconds after electricity reading 30000 was taken.

The data logger starts the logging processes simultaneously with a view to obtaining concurrent readings and events from the live data feed.  However, the timestamp on each record reflects the time the record was written to the database file.  This indicates that retrieving the ANM log takes longer than retrieving an electricity reading. The result is the mismatch of records shown above where, clearly, the electricity reading should be matched to the ANM log that occurs a few seconds later.

This can be achieved by perfoming the search using truncated timestamps - ommitting the 'seconds' field.

In [9]:
# match most relevant ANM log event to a reading from the electricity log, where the ANM event is either the most recent or occurs within the same minute period.
# this is done by performing the search truncating the timestamps to exclude 'seconds' 

reading = readings_orig[30000]
readingdate= "'" + reading['timestamp'] + "'"
anmstatus = %sql select * from records where substr(timestamp, 0, 17) <= substr($readingdate, 0, 17) order by timestamp desc limit 1;
print(reading['timestamp'], anmstatus[0]['timestamp'])

 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:14:09 2019-02-06 18:14:13


In [12]:
import json

In [14]:
for reading in readings_orig[30000:30010]:
    readingdate= "'"+reading['timestamp']+"'"
    anmrecord = %sql select * from records where substr(timestamp, 0, 17) <= substr($readingdate, 0, 17) order by timestamp desc limit 1;
    anmstatus = json.loads(anmrecord[0]['log'])
    print(reading['timestamp'],
          reading['demand'],
          reading['demand_max'],
          reading['anm_generated'],
          reading['non_anm_generated'],
          reading['total_generation_capacity'],
          anmrecord[0]['timestamp'],
          anmstatus['Core'][0],
          anmstatus['Core'][1],
          anmstatus['Core'][2],
          anmstatus['1'][0],
          anmstatus['1'][1],
          anmstatus['1'][2],
          anmstatus['2'][0],
          anmstatus['2'][1],
          anmstatus['2'][2],
          anmstatus['2A'][0],
          anmstatus['2A'][1],
          anmstatus['2A'][2],
          anmstatus['2B'][0],
          anmstatus['2B'][1],
          anmstatus['2B'][2],
          anmstatus['3'][0],
          anmstatus['3'][1],
          anmstatus['3'][2],
          anmstatus['4'][0],
          anmstatus['4'][1],
          anmstatus['4'][2],
          anmstatus['4A'][0],
          anmstatus['4A'][1],
          anmstatus['4A'][2])
    
    

 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:14:09 21.62 35.7 7.592 14.065 57.099999999999994 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:15:05 21.88 35.7 10.149 14.043 57.1 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:16:05 21.59 35.7 9.881 12.069 57.1 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
 * sqlite:///./database/combined.db
   sqlite:///./database/eleclog_copy.db
Done.
2019-02-06 18:17:05 21.67 35.7 9.849 12.467 57.099999999999994 2019-02-06 18:14:13 ok ok ok ok ok ok warning ok ok warning ok ok ok ok ok ok ok ok ok ok ok ok ok ok
 * sqlite:///./database/combined.db
   